In [2]:
from os.path import abspath, join
import os
import pickle

import pandas as pd
import numpy as np

%matplotlib inline

## deal with password
- https://tianchi.aliyun.com/forum/postDetail?spm=5176.12586969.1002.39.6c3f29e8fpdvQ6&postId=102089

In [3]:
df_pass = pd.read_clipboard()

In [4]:
df_pass

,Phase,Filename,Password
0,T=0,underexpose_train_click-0,NaN
1,T=0,underexpose_test_click-0,NaN
2,T=1,underexpose_train_click-1,7c2d2b8a636cbd790ff12a007907b2ba
3,T=1,underexpose_test_click-1,96d071a532e801423be614e9e8414992
4,T=2,underexpose_train_click-2,ea0ec486b76ae41ed836a8059726aa85
5,T=2,underexpose_test_click-2,503bf7a5882d3fac5ca9884d9010078c
6,T=3,underexpose_train_click-3,65255c3677a40bf4d341b0c739ad6dff
7,T=3,underexpose_test_click-3,dd3de82d0b3a7fe9c55e0b260027f50f
8,T=4,underexpose_train_click-4,c8376f1c4ed07b901f7fe5c60362ad7b
9,T=4,underexpose_test_click-4,04e966e4f6c7b48f1272a53d8f9ade5d


In [5]:
df_pass.iloc[12, 1]  = 'underexpose_train_click-6'
df_pass.iloc[13, 1]  = 'underexpose_test_click-6'

In [6]:
df_pass

,Phase,Filename,Password
0,T=0,underexpose_train_click-0,NaN
1,T=0,underexpose_test_click-0,NaN
2,T=1,underexpose_train_click-1,7c2d2b8a636cbd790ff12a007907b2ba
3,T=1,underexpose_test_click-1,96d071a532e801423be614e9e8414992
4,T=2,underexpose_train_click-2,ea0ec486b76ae41ed836a8059726aa85
5,T=2,underexpose_test_click-2,503bf7a5882d3fac5ca9884d9010078c
6,T=3,underexpose_train_click-3,65255c3677a40bf4d341b0c739ad6dff
7,T=3,underexpose_test_click-3,dd3de82d0b3a7fe9c55e0b260027f50f
8,T=4,underexpose_train_click-4,c8376f1c4ed07b901f7fe5c60362ad7b
9,T=4,underexpose_test_click-4,04e966e4f6c7b48f1272a53d8f9ade5d


In [7]:
# pass dict
dict_pass = df_pass.set_index("Filename")["Password"].to_dict()

In [9]:
TRAIN_DATA = "../data/raw/underexpose_train"
TEST_DATA = "../data/raw/underexpose_test"
UNZIP_DIR  = abspath("../data/raw/")

OUTPUT_DIR = "../data/out/"

In [13]:
if 1 == 0:
    ALL_DATA_FP = abspath(join(UNZIP_DIR, "all_data.zip"))
    UNZIP_CMD = f"unzip {ALL_DATA_FP} -d {UNZIP_DIR}"
    r = os.system(UNZIP_CMD)
    print(r)

256


In [14]:
if 1 == 1:
    for file_name, pass_ in dict_pass.items():
        if pass_ is not np.nan and pass_ != "TBA":
            if "train" in file_name:
                DIR_NAME = TRAIN_DATA
            else:
                DIR_NAME = TEST_DATA

            file_path = abspath(join(DIR_NAME, file_name + ".zip"))  
            dest_path = abspath(join(DIR_NAME))     
            unzip_cmd = "unzip -P {pass_} {zip_file} -d {dest_path}".format(pass_=pass_, zip_file=file_path, dest_path=dest_path)
            r = os.system(unzip_cmd)
            print(r)

0
0
0
0
0
0
0
0
0
0
0
0


## reading all pharse data

In [28]:
if 1 == 1:
    data_mapping = {
        "item_feat": (join(TRAIN_DATA, "underexpose_item_feat.csv"), ["item_id", "txt_vec", "img_vec"]),
        "user_feat": (join(TRAIN_DATA, "underexpose_user_feat.csv"), ["user_id", "user_age_level", "user_gender", "user_city_level"]),
        "train_click": (join(TRAIN_DATA, "underexpose_train_click-{}.csv"), ["user_id", "item_id", "time"]),
        "test_click": (join(TEST_DATA, "underexpose_test_click-{}.csv"), ["user_id", "item_id", "time"]),
        "test_qtime": (join(TEST_DATA, "underexpose_test_qtime-{}.csv"), ["user_id", "query_time"])
    }
    
    
    def read_csv(k, p, cols):
        if k == "item_feat":
            return pd.read_csv(p, header=None, names=cols, sep=",\[")
        if "train" in k or "test" in k:
            data = []
            for i in range(7):
                data.append(pd.read_csv(p.format(i), header=None, names=cols).assign(Phase=i))
            return pd.concat(data, axis=0)
        return pd.read_csv(p, header=None, names=cols) 
    
    df_data_mapping = {
        k: read_csv(k, p, cols) for k, (p, cols) in data_mapping.items()
    }

    df_data_mapping["item_feat"]["txt_vec"] = df_data_mapping["item_feat"]["txt_vec"].apply(lambda x: x[:-1])
    df_data_mapping["item_feat"]["img_vec"] = df_data_mapping["item_feat"]["img_vec"].apply(lambda x: x[:-1])

    for k, df in df_data_mapping.items():
        print(f"{k}:")
        display(df.head(3))
        display(df.tail(3))

/Users/tedzhao/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


item_feat:


,item_id,txt_vec,img_vec
0,42844,"4.514945030212402, -2.3837196826934814, 0.5004...","-2.8722801208496094, 1.4587551355361938, 2.579..."
1,67898,"-2.0029051303863525, -0.9298805594444275, 0.79...","-0.07052088528871536, -1.4393335580825806, 0.7..."
2,66446,"4.221673011779785, -1.4971394538879395, 1.1335...","-5.18036413192749, -0.38824713230133057, -0.03..."


,item_id,txt_vec,img_vec
108913,62184,"1.4589283466339111, -0.6386770606040955, -0.95...","-1.8863677978515625, 0.2406638264656067, -0.41..."
108914,42748,"3.3835203647613525, -1.6698633432388306, 1.264...","-2.6400721073150635, 1.0721999406814575, 1.514..."
108915,61098,"2.4478535652160645, -0.46994200348854065, 1.87...","-1.1794008016586304, -0.6624383926391602, 1.38..."


user_feat:


...

,user_id,item_id,time,Phase
0,1133,221,0.983812,0
1,17864,253,0.983783,0
2,6941,309,0.983785,0


,user_id,item_id,time,Phase
28785,18057,117396,0.984172,6
28786,6034,117427,0.984112,6
28787,2943,117431,0.984094,6


test_qtime:


,user_id,query_time,Phase
0,11,0.983869,0
1,22,0.983956,0
2,44,0.983924,0


,user_id,query_time,Phase
1818,35415,0.984284,6
1819,35426,0.984266,6
1820,35437,0.984262,6


In [29]:
# save data as pickle

if 1 == 1:
    with open(join(OUTPUT_DIR, "data.pkl"), "wb") as file:
        pickle.dump(df_data_mapping, file)
else:
    with open(join(OUTPUT_DIR, "data.pkl"), "rb") as file:
        df_data_mapping = pickle.load(file)